Importing all relevant libraries and sql

In [ ]:
import psycopg2
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import scipy.stats as stats 
import seaborn as sns 
from IPython.display import display

Step 1: Analyzing the relationship between production budgets and metascore to see how it influences movie box_office

In [ ]:
#establishing a database function  Note: The real credentials are hidden
def connectDB(): 
    try: #using try and except to make sure if there is an error to return 'database connection error'
        conn = psycopg2.connect(
        host = 'abcd',
        port = 'abcd',
        database = 'abcd',
        user = 'abcd',
        password = 'abcd')
        print ("Database connection succesfull")
        return conn
    except psycopg2.Error as e:
        print(f"Database connection error: {e}")
        return None

#connecting to the database
conn = connectDB() 

#setting cursor to execute commands 
cur = conn.cursor()


#using execute statement to create a temporary table based on SQL schema
cur.execute(""" CREATE TEMP TABLE budget_and_metascore AS
WITH ranked_movies AS ( 
	SELECT
		m.title,
		CAST(s.box_office AS NUMERIC) AS budget, 
		CAST(m.metascore AS NUMERIC) AS metascore, 
		CAST(s.box_office AS NUMERIC) AS box_office, 
		g.genre,
			RANK() OVER (PARTITION BY g.genre ORDER BY s.box_office DESC) AS box_office_rank 
		FROM 
			movie AS m
		INNER JOIN
			sales AS s ON m.title = s.title
		LEFT JOIN
			movie_genre AS g ON m.title = g.title
) 

SELECT
    genre,
    AVG(CASE WHEN box_office_rank <= 100 THEN budget END) AS avg_top_budget,
    AVG(CASE WHEN box_office_rank > 100 THEN budget END) AS avg_low_budget,
	AVG(CASE WHEN box_office_rank <= 100 THEN metascore END) AS avg_top_metascore,
    AVG(CASE WHEN box_office_rank > 100 THEN metascore END) AS avg_low_metascore,
	AVG(CASE WHEN box_office_rank <= 100 THEN box_office END) AS avg_box_office_top_100,
    AVG(CASE WHEN box_office_rank > 100 THEN box_office END) AS avg_box_office_low_performing
FROM
    ranked_movies

GROUP BY
	genre 
ORDER BY 
	avg_top_budget DESC, avg_top_metascore DESC, avg_low_budget DESC, avg_low_metascore DESC, avg_box_office_top_100 DESC, avg_box_office_low_performing DESC;

""")

#additional execute to Select from the temporary table to be able to continue with the caluclation 
cur.execute("""SELECT avg_box_office_top_100 AS box_office, avg_top_metascore AS metascore, avg_top_budget AS production_budget FROM budget_and_metascore""")

#Fetch all data from the result set into a dataframe
data = pd.DataFrame(cur.fetchall(), columns=["box_office", "metascore", "production_budget"])

#Converting all the columns to float using astype function
data['box_office'] = data['box_office'].astype(float) 
data['metascore'] = data['metascore'].astype(float)
data['production_budget'] = data['production_budget'].astype(float)

#further more we need to delete all the missing values. We drop NaN values from the selected table/attributes

data.dropna(inplace=True)

#calculating correlation coefficients 

correlation_box_office_metacore = data['box_office'].corr(data['metascore'])
correlation_box_office_budget = data['box_office'].corr(data['production_budget'])
correlation_metascore_budget = data['metascore'].corr(data['production_budget'])

#We will also print the correlation coefficinets results 

print("Correlation between Box Office and Metascore", correlation_box_office_metacore)
print("Correlation between Box Office and Production Budget", correlation_box_office_budget)
print("Correlation between Metascore and Production Budget", correlation_metascore_budget)




#Commit needed to commit to the database
conn.commit() 

#at the end of our code we need to close the cursor and connection with the database 
cur.close()
conn.close() 

#furthermore we can use this information for further research purposes and interperet our correlation coeficient results. In additon we can use seaborn library to visualaize the results 

Step 2: Analyzing the relationship between director reputation, as measured by number of awards received and reputation score, and sales performance measured by sales margin percentage of movies

In [ ]:
# Create a cursor object to interact with the database
cur = conn.cursor()


# Execute queries within the same session
# Creating the temporary table as it is in SQL
cur.execute("""CREATE TEMP TABLE director_reputation AS
    SELECT
        m.title, 
        m.director, 
        CAST(m.release_year AS INTEGER) AS release_year,
        s.genre,
        CAST(s.box_office AS NUMERIC) AS box_office,
        CAST(s.production_budget AS NUMERIC) AS production_budget,
            CASE 
                WHEN CAST(s.box_office AS NUMERIC) = 0 THEN NULL
                    ELSE (CAST(s.box_office AS NUMERIC) - CAST(s.production_budget AS NUMERIC)) / NULLIF(CAST(s.box_office AS NUMERIC), 0) * 100
                END AS sales_margin_percentage,
        d.avg_expert_score,
        
        COUNT(a.awards) AS total_awards,
        COUNT(*) OVER (PARTITION BY m.director) AS movie_count_per_director
        
    FROM
        movie AS m
    INNER JOIN
        sales AS s
        ON m.title = s.title
    LEFT JOIN
        ( 
            SELECT
                m.director,
                AVG(CAST(e.idvscore AS DECIMAL(10, 2))) AS avg_expert_score
            FROM
                movie AS m
            LEFT JOIN
                expert_review AS e
                ON m.url = e.url
            GROUP BY
                m.director
        ) AS d
        ON m.director = d.director
    LEFT JOIN
        movie_awards AS a
        ON m.title = a.title
    GROUP BY
        m.title, m.director, m.metascore, m.userscore, s.genre, s.box_office, s.production_budget, d.avg_expert_score
    ORDER BY
        box_office DESC;            
""")  

conn.commit() # Commit the changes to the database 


# Execute the query to select data from the temporary table
cur.execute("SELECT sales_margin_percentage, avg_expert_score FROM director_reputation")

# Fetch all rows from the result set into a Pandas DataFrame
data = pd.DataFrame(cur.fetchall(), columns=["sales_margin_percentage", "avg_expert_score"])


# Convert 'sales_margin_percentage' and 'avg_expert_score' columns to float
data['sales_margin_percentage'] = data['sales_margin_percentage'].astype(float)
data['avg_expert_score'] = data['avg_expert_score'].astype(float)


#Drop rows with NaN values in either of the columns
data.dropna(subset=["sales_margin_percentage", "avg_expert_score"], inplace=True)


# Calculate the correlation coefficient and p-value
correlation, p_value = pearsonr(data["sales_margin_percentage"], data["avg_expert_score"])

# Print the correlation coefficient and p-value
print("Correlation Coefficient:", correlation)
print("P-value:", p_value)

    
#Close the cursor and database connection
cur.close()
conn.close()

#The data can then be used for further research to interpret the results based on the correlation coefficient and p-value

Step 3: Analyzing the impact of movie genre produced by studios specialized in that genre can have on movie sales.

We will rank movie genres by profit per genre to have the top performing genres and then populate the studio and release month

In [ ]:
# Create a cursor to connect to the database
cur = conn.cursor()


# Execute queries within the same session
# Creating the temporary table as it is in SQL
cur.execute("""CREATE TEMP TABLE studio_genre AS
WITH movies_profit AS (
    SELECT
        s.genre,
        m.studio,
        TO_CHAR(TO_DATE(s.release_date, 'DD/MM/YYYY'), 'Month') AS release_month,

        SUM(CAST(s.box_office AS numeric)) - SUM(CAST(s.production_budget AS numeric)) AS profit
      
    FROM
        sales s
    JOIN
        movie m
    ON
        s.title = m.title
    GROUP BY
        s.genre,
        m.studio,
        release_month
)

SELECT
    genre,
    studio,
    release_month,
    profit
FROM (
    SELECT
        genre,
        studio,
        release_month,
        profit,
        RANK() OVER (PARTITION BY genre ORDER BY profit DESC) AS genre_ranking
    FROM
        movies_profit
) ranking
WHERE
    genre_ranking = 1
ORDER BY
    profit DESC;
""")

# Commit the changes
conn.commit()

# Use pandas to get the data and convert to a DataFrame
df = pd.read_sql_query("SELECT * FROM studio_genre", conn)

# Use pandas to display numbers in standard format
pd.set_option('display.float_format', '{:.0f}'.format)  

# Display the DataFrame as a graphic/table
display(df)

# Calculate mean and median for the 'profit' column
mean_profit = df['profit'].mean()
median_profit = df['profit'].median()

# Print the mean and median profit
print("Mean Profit:", mean_profit)
print("Median Profit:", median_profit)

# Close the cursor and connection
cur.close()
conn.close()

Step 4: Analyzing the relationship between movie sales measured by box office collection to that of the average idvscore for user review and expert review

In [ ]:
# Create a cursor object to interact with the database
cur = conn.cursor()


# Execute queries within the same session
# Creating the temporary table as it is in SQL
cur.execute("""CREATE TEMP TABLE boxoffice_reputation AS
WITH Top100HighestBoxOffice AS (
	SELECT
    s.title AS movie_name, 
    AVG(CAST(ur.idvscore AS NUMERIC)) AS "avg_user_review_idv_score",   
    AVG(CAST(er.idvscore AS NUMERIC)) AS "avg_expert_review_idv_score"  
FROM
    movie AS m
JOIN
    user_review AS ur ON m.url = ur.url
JOIN
    expert_review AS er ON m.url = er.url
JOIN
    sales AS s ON m.title = s.title
GROUP BY
    s.title
ORDER BY
    MAX(CAST(s.box_office AS NUMERIC)) DESC  
LIMIT
    100
),	


Top100LowestBoxOffice AS ( 
	SELECT
    s.title AS movie_name, 
    AVG(CAST(ur.idvscore AS NUMERIC)) AS "avg_user_review_idv_score",   
    AVG(CAST(er.idvscore AS NUMERIC)) AS "avg_expert_review_idv_score"  
FROM
    movie AS m
JOIN
    user_review AS ur ON m.url = ur.url
JOIN
    expert_review AS er ON m.url = er.url
JOIN
    sales AS s ON m.title = s.title
WHERE CAST(s.box_office AS NUMERIC) > 0  
GROUP BY
    s.title
ORDER BY
    MIN(CAST(s.box_office AS NUMERIC)) ASC
LIMIT
    100
),

CombinedResults AS (
    SELECT 'Top100HighestBoxOffice' AS BoxOffice,     
           movie_name,
           avg_user_review_idv_score,
           avg_expert_review_idv_score
    FROM Top100HighestBoxOffice
    UNION ALL
    SELECT 'Top100LowestBoxOffice' AS BoxOffice,      
           movie_name,
           avg_user_review_idv_score,
           avg_expert_review_idv_score
    FROM Top100LowestBoxOffice
)

SELECT BoxOffice,
       AVG(avg_user_review_idv_score) AS overall_avg_user_review,
       AVG(avg_expert_review_idv_score) AS overall_avg_expert_review
FROM CombinedResults
GROUP BY BoxOffice
ORDER BY overall_avg_user_review DESC;
""")  

conn.commit() # Commit the changes to the database 


# Execute the query to select data from the temporary table
cur.execute("SELECT * FROM boxoffice_reputation")

# Fetch all rows from the result set into a Pandas DataFrame
data = pd.DataFrame(cur.fetchall(), columns=["BoxOffice","overall_avg_user_review", "overall_avg_expert_review"])

# Display the DataFrame as a graphic/table
display(data)

    
#Close the cursor and database connection
cur.close()
conn.close()

#The data can then be used for further research to interpret the results based on the research & proceed to do some sentiment analysis of expert review text. 